In [ ]:
import tkinter as tk
import numpy as np
import random
import time
from queue import Queue, PriorityQueue

CANVAS_SIZE = 600
PUZZLE_GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]  # Goal state for 8-puzzle

class MazeSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Search Problem Solver")

        # Canvas setup
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=6)

        # Control buttons
        self.random_button = tk.Button(self.root, text="Random", command=self.generate_random_state)
        self.random_button.grid(row=1, column=0)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=self.solve_with_dfs)
        self.dfs_button.grid(row=1, column=1)

        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=self.solve_with_bfs)
        self.bfs_button.grid(row=1, column=2)

        self.astar_button = tk.Button(self.root, text="Solve with A*", command=self.solve_with_astar)
        self.astar_button.grid(row=1, column=3)

        self.problem_var = tk.StringVar(value="8-Puzzle")
        self.problem_menu = tk.OptionMenu(self.root, self.problem_var, "8-Puzzle", command=self.change_problem)
        self.problem_menu.grid(row=1, column=4)

        # Initialize 8-puzzle
        self.puzzle_start = None
        self.solution_path = None
        self.generate_random_state()

    def change_problem(self, problem):
        """Handle problem change (currently only 8-puzzle)."""
        if problem == "8-Puzzle":
            self.generate_random_state()

    def generate_random_state(self):
        """Generate a random state for 8-puzzle."""
        state = list(range(9))
        random.shuffle(state)
        self.puzzle_start = state
        self.draw_puzzle()

    def draw_puzzle(self):
        """Draw the 8-puzzle on the canvas."""
        size = int(len(self.puzzle_start) ** 0.5)
        cell_size = CANVAS_SIZE // size
        self.canvas.delete("all")

        for i in range(size):
            for j in range(size):
                value = self.puzzle_start[i * size + j]
                color = 'white' if value != 0 else 'black'
                self.canvas.create_rectangle(j * cell_size, i * cell_size,
                                             (j + 1) * cell_size, (i + 1) * cell_size, fill=color)
                if value != 0:
                    self.canvas.create_text(j * cell_size + cell_size / 2, i * cell_size + cell_size / 2,
                                            text=str(value), font=("Arial", 20))

    def solve_with_dfs(self):
        """Solve 8-puzzle using DFS."""
        self.solve_problem(dfs_puzzle, "DFS")

    def solve_with_bfs(self):
        """Solve 8-puzzle using BFS."""
        self.solve_problem(bfs_puzzle, "BFS")

    def solve_with_astar(self):
        """Solve 8-puzzle using A*."""
        self.solve_problem(a_star_puzzle, "A*")

    def solve_problem(self, algorithm, name):
        start_time = time.time()
        self.solution_path = algorithm(self.puzzle_start, PUZZLE_GOAL_STATE)

        if self.solution_path:
            print(f"{name} solved the problem in {time.time() - start_time:.4f} seconds.")
            self.display_puzzle_solution()
        else:
            print(f"{name} could not find a solution.")

    def display_puzzle_solution(self):
        """Animate the solution path on the canvas."""
        for state in self.solution_path:
            self.puzzle_start = state
            self.draw_puzzle()
            self.root.update()
            time.sleep(0.5)

# Algorithms for 8-Puzzle
def dfs_puzzle(start, goal):
    """DFS algorithm for 8-puzzle."""
    stack = [(start, [])]
    visited = set()

    while stack:
        state, path = stack.pop()
        if state == goal:
            return path + [state]

        visited.add(tuple(state))
        for neighbor in generate_neighbors(state):
            if tuple(neighbor) not in visited:
                stack.append((neighbor, path + [state]))

    return None

def bfs_puzzle(start, goal):
    """BFS algorithm for 8-puzzle."""
    queue = Queue()
    queue.put((start, []))
    visited = set()

    while not queue.empty():
        state, path = queue.get()
        if state == goal:
            return path + [state]

        visited.add(tuple(state))
        for neighbor in generate_neighbors(state):
            if tuple(neighbor) not in visited:
                queue.put((neighbor, path + [state]))

    return None

def a_star_puzzle(start, goal):
    """A* algorithm for 8-puzzle."""
    pq = PriorityQueue()
    pq.put((0, start, []))
    visited = set()

    while not pq.empty():
        _, state, path = pq.get()
        if state == goal:
            return path + [state]

        visited.add(tuple(state))
        for neighbor in generate_neighbors(state):
            if tuple(neighbor) not in visited:
                cost = len(path) + heuristic_puzzle(neighbor, goal)
                pq.put((cost, neighbor, path + [state]))

    return None

def generate_neighbors(state):
    """Generate valid neighbors for the 8-puzzle state."""
    neighbors = []
    zero_index = state.index(0)
    x, y = divmod(zero_index, 3)

    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_state = state[:]
            swap_index = nx * 3 + ny
            new_state[zero_index], new_state[swap_index] = new_state[swap_index], new_state[zero_index]
            neighbors.append(new_state)

    return neighbors

def heuristic_puzzle(state, goal):
    """Manhattan distance heuristic for A*."""
    distance = 0
    for i in range(1, 9):
        x1, y1 = divmod(state.index(i), 3)
        x2, y2 = divmod(goal.index(i), 3)
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = MazeSolverApp(root)
    root.mainloop()
